The archive contains the files data_batch_1, data_batch_2, ..., data_batch_5, as well as test_batch. Each of these files is a Python "pickled" object produced with cPickle. 

** Load the Data. Use the Code Below to load the data: **

In [4]:
TRAIN_PATH = 'train_amex/train.csv'

In [176]:
import pandas as pd
import numpy as np
import tensorflow as tf
dataset =  pd.read_csv(TRAIN_PATH)
print(dataset.columns)

Index(['session_id', 'DateTime', 'user_id', 'product', 'campaign_id',
       'webpage_id', 'product_category_1', 'product_category_2',
       'user_group_id', 'gender', 'age_level', 'user_depth',
       'city_development_index', 'var_1', 'is_click'],
      dtype='object')


In [536]:
val_frame = dataset[['DateTime','product','webpage_id','product_category_1','product_category_2','campaign_id','user_group_id','gender','age_level','user_depth','city_development_index','var_1','is_click']]

In [537]:

print(val_frame.head)

print(val_frame['product'].unique())
val_temp = val_frame

<bound method NDFrame.head of                 DateTime product  webpage_id  product_category_1  \
0       2017-07-02 00:00       C       13787                   4   
1       2017-07-02 00:00       C       11085                   5   
2       2017-07-02 00:00       C       13787                   4   
3       2017-07-02 00:00       I       13787                   3   
4       2017-07-02 00:01       C       60305                   3   
5       2017-07-02 00:01       I       13787                   2   
6       2017-07-02 00:01       F       60305                   3   
7       2017-07-02 00:01       C       13787                   3   
8       2017-07-02 00:02       H       11085                   5   
9       2017-07-02 00:02       B       13787                   2   
10      2017-07-02 00:02       C       60305                   3   
11      2017-07-02 00:02       H       53587                   1   
12      2017-07-02 00:02       H       13787                   3   
13      2017-07-02

In [538]:
val_temp['DateTime'] = val_temp['DateTime'].apply(lambda x: (str(x)[11:13]))

/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [541]:
val_temp['campaign_id'].unique()

array(['359520', '105960', '405490', '360936', '404347', '98970',
       '414149', '82320', '396664', '118601'], dtype=object)

In [540]:
val_temp.age_level = val_temp.age_level.apply(str)
val_temp.user_depth = val_temp.user_depth.apply(str)
val_temp.city_development_index = val_temp.city_development_index.apply(str)
val_temp.product_category_1 = val_temp.product_category_1.apply(str)
val_temp.user_group_id = val_temp.user_group_id.apply(str)
val_temp.webpage_id = val_temp.webpage_id.apply(str)
val_temp.var_1 = val_temp.var_1.apply(str)
val_temp.campaign_id = val_temp.campaign_id.apply(str)

/anaconda3/envs/tensorflow/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [542]:
print(len(val_temp))
val_temp = val_temp.dropna(subset=['gender','product'])
print(len(val_temp))
val_temp = val_temp[val_temp.age_level != 'nan' ]
val_temp = val_temp[val_temp.user_depth != 'nan']
val_temp = val_temp[val_temp.city_development_index!= 'nan']
val_temp = val_temp[val_temp.product_category_1 != 'nan']
val_temp = val_temp[val_temp.user_group_id != 'nan']

print(len(val_temp))

463291
445048
338162


In [544]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender" , ["Female","Male"])
product = tf.feature_column.categorical_column_with_vocabulary_list("product",['C' 'I' 'F' 'H' 'B' 'D' 'G' 'E' 'J' 'A'])
DateTime = tf.feature_column.categorical_column_with_vocabulary_list("DateTime",['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10','11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21','22', '23'])
product_category_1 = tf.feature_column.categorical_column_with_vocabulary_list("product_category_1",['4', '5', '3', '2', '1'])
age_level = tf.feature_column.categorical_column_with_vocabulary_list("age_level",['4.0', '2.0', '3.0', '1.0', '5.0', '6.0', '0.0'])
user_depth = tf.feature_column.categorical_column_with_vocabulary_list("user_depth", ['3.0', '2.0','1.0'])
city_development_index = tf.feature_column.categorical_column_with_vocabulary_list("city_development_index", ['3.0', '2.0', '4.0', '1.0'])
user_group_id = tf.feature_column.categorical_column_with_vocabulary_list("user_group_id",['10.0', '3.0', '2.0', '9.0', '4.0', '11.0', '1.0', '7.0', '5.0','8.0', '12.0', '6.0', '0.0'])
webpage_id = tf.feature_column.categorical_column_with_vocabulary_list("webpage_id",['13787', '11085', '60305', '53587', '6970', '45962', '1734','51181', '28529'])
var_1 = tf.feature_column.categorical_column_with_vocabulary_list("var_1", ['0', '1'])
campaign_id = tf.feature_column.categorical_column_with_vocabulary_list("campaign_id",['359520', '105960', '405490', '360936', '404347', '98970','414149', '82320', '396664', '118601'])

g_e = tf.feature_column.embedding_column(gender,dimension=2)
p_e = tf.feature_column.embedding_column(product,dimension=10)
dt_e = tf.feature_column.embedding_column(DateTime,dimension=24)
pc_1 = tf.feature_column.embedding_column(product_category_1,dimension=5)
a_l = tf.feature_column.embedding_column(age_level,dimension=5)
u_d = tf.feature_column.embedding_column(user_depth,dimension=3)
c_d_e = tf.feature_column.embedding_column(city_development_index,dimension=4)
u_g_i = tf.feature_column.embedding_column(user_group_id,dimension=13)
w_i = tf.feature_column.embedding_column(webpage_id,dimension=9)
v_1 = tf.feature_column.embedding_column(var_1,dimension=2)
c_i = tf.feature_column.embedding_column(campaign_id,dimension=10)


In [621]:
feat_cols = [gender,product,DateTime,webpage_id,product_category_1,age_level,user_depth,city_development_index]

feat_cols_e = [g_e,p_e,dt_e,pc_1,a_l,u_d,c_d_e,u_g_i,w_i]

In [622]:
print(len(val_temp[val_temp.is_click == 1]))
len(val_temp[val_temp.is_click == 1])/len(val_temp)*100

positive_samples = val_temp[val_temp.is_click == 1]
negative_samples = val_temp[val_temp.is_click == 0]

22849


** Why the 'b's in front of the string? **
Bytes literals are always prefixed with 'b' or 'B'; they produce an instance of the bytes type instead of the str type. They may only contain ASCII characters; bytes with a numeric value of 128 or greater must be expressed with escapes.

https://stackoverflow.com/questions/6269765/what-does-the-b-character-do-in-front-of-a-string-literal

In [623]:
negative_samples = negative_samples.sample(n=31331)
    

In [624]:
frames = [negative_samples,positive_samples]
df = pd.concat(frames)
df.head()

,DateTime,product,webpage_id,product_category_1,product_category_2,campaign_id,user_group_id,gender,age_level,user_depth,city_development_index,var_1,is_click
177556,10,F,53587,1,NaN,404347,5.0,Male,5.0,3.0,2.0,0,0
126560,14,E,53587,1,146115.0,404347,11.0,Female,5.0,3.0,2.0,1,0
9782,07,C,13787,4,NaN,359520,2.0,Male,2.0,3.0,3.0,0,0
208952,18,H,13787,4,NaN,359520,3.0,Male,3.0,3.0,4.0,1,0
233800,00,H,13787,2,NaN,359520,4.0,Male,4.0,3.0,3.0,0,0


In [625]:
from sklearn.utils import shuffle
df = shuffle(df)
t_data = val_temp.sample(n=128858) 
xd_data = t_data.drop('is_click',axis=1)
yd_data = t_data['is_click']

In [626]:
X_data = df.drop('is_click',axis=1)
Y_data = df['is_click']

In [627]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X_data,Y_data,test_size= 0.2, random_state=101)

In [628]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_data,y=Y_data,batch_size=10, num_epochs=100,shuffle=True)

In [629]:
model = tf.estimator.DNNClassifier(feature_columns=feat_cols_e,
                                     hidden_units=[512,512,512],
                                     optimizer=tf.train.AdamOptimizer(0.0001),
                                     n_classes=2,dropout = 0.1)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/pt/g1fbn1491ls6jzjxv526n9j40000gn/T/tmpeknsdldo', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb37ec4860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [630]:
model.train(input_func,steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/pt/g1fbn1491ls6jzjxv526n9j40000gn/T/tmpeknsdldo/model.ckpt.
INFO:tensorflow:loss = 7.100932, step = 1
INFO:tensorflow:global_step/sec: 78.908
INFO:tensorflow:loss = 5.8015213, step = 101 (1.269 sec)
INFO:tensorflow:global_step/sec: 223.06
INFO:tensorflow:loss = 7.00259, step = 201 (0.448 sec)
INFO:tensorflow:global_step/sec: 225.011
INFO:tensorflow:loss = 6.4237585, step = 301 (0.445 sec)
INFO:tensorflow:global_step/sec: 228.306
INFO:tensorflow:loss = 7.260347, step = 401 (0.438 sec)
INFO:tensorflow:global_step/sec: 228.442
INFO:tensorflow:loss = 7.6091433, step = 501 (0.437 sec)
INFO:tensorflow:global_step/sec: 224.676
INFO:tensorflow:loss = 6.556451, step = 601 (0.446 sec)
INFO:tensorflow:glob

INFO:tensorflow:loss = 6.4821987, step = 8001 (0.462 sec)
INFO:tensorflow:global_step/sec: 210.474
INFO:tensorflow:loss = 6.9883904, step = 8101 (0.475 sec)
INFO:tensorflow:global_step/sec: 229.481
INFO:tensorflow:loss = 5.3380513, step = 8201 (0.436 sec)
INFO:tensorflow:global_step/sec: 175.111
INFO:tensorflow:loss = 6.2768626, step = 8301 (0.571 sec)
INFO:tensorflow:global_step/sec: 220.984
INFO:tensorflow:loss = 7.2521043, step = 8401 (0.452 sec)
INFO:tensorflow:global_step/sec: 215.523
INFO:tensorflow:loss = 4.503779, step = 8501 (0.464 sec)
INFO:tensorflow:global_step/sec: 226.844
INFO:tensorflow:loss = 6.8453646, step = 8601 (0.441 sec)
INFO:tensorflow:global_step/sec: 216.99
INFO:tensorflow:loss = 7.0865164, step = 8701 (0.461 sec)
INFO:tensorflow:global_step/sec: 169.699
INFO:tensorflow:loss = 7.37601, step = 8801 (0.590 sec)
INFO:tensorflow:global_step/sec: 222.857
INFO:tensorflow:loss = 6.915947, step = 8901 (0.449 sec)
INFO:tensorflow:global_step/sec: 207.677
INFO:tensorflow

In [631]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=xd_data,batch_size=len(xd_data),shuffle=False)
tfn = tf.estimator.inputs.pandas_input_fn(x=xd_data,batch_size=len(xd_data),shuffle=False)

In [632]:
predictions = list(model.predict(input_fn=tfn))
predictions

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/pt/g1fbn1491ls6jzjxv526n9j40000gn/T/tmpeknsdldo/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'logits': array([0.12289812], dtype=float32),
  'logistic': array([0.5306859], dtype=float32),
  'probabilities': array([0.46931407, 0.5306859 ], dtype=float32),
  'class_ids': array([1]),
  'classes': array([b'1'], dtype=object)},
 {'logits': array([-0.44361052], dtype=float32),
  'logistic': array([0.390881], dtype=float32),
  'probabilities': array([0.609119  , 0.39088097], dtype=float32),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([-0.27139163], dtype=float32),
  'logistic': array([0.4325655], dtype=float32),
  'probabilities': array([0.5674345 , 0.43256548], dtype=float32),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([-0.39696518], dtype=float32),
  'logistic': array([0.4020417], dtype=float32),
  'probabilities': array([0.59795827, 0.4020417 ], dtype=float32),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([-0.2649814], dtype=float32),
  'logistic': 

# Helper Functions for Dealing With Data.

** Use the provided code below to help with dealing with grabbing the next batch once you've gotten ready to create the Graph Session. Can you break down how it works? **

In [633]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [634]:
from sklearn.metrics import roc_auc_score

** How to use the above code: **

In [635]:
print(roc_auc_score(yd_data,final_preds))

0.5402893727341364


## Creating the Model

** Import tensorflow **

In [514]:
from sklearn.linear_model import LogisticRegression

** Create 2 placeholders, x and y_true. Their shapes should be: **

* x shape = [None,32,32,3]
* y_true shape = [None,10]


In [515]:
model = LogisticRegression()
model = model.fit(X_train,y_train)

/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: could not convert string to float: 'Male'

** Create one more placeholder called hold_prob. No need for shape here. This placeholder will just hold a single probability for the dropout. **

In [22]:
hold_prob = tf.placeholder(tf.float32)

### Helper Functions

** Grab the helper functions from MNIST with CNN (or recreate them here yourself for a hard challenge!). You'll need: **

* init_weights
* init_bias
* conv2d
* max_pool_2by2
* convolutional_layer
* normal_full_layer

In [23]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

### Create the Layers

** Create a convolutional layer and a pooling layer as we did for MNIST. **
** Its up to you what the 2d size of the convolution should be, but the last two digits need to be 3 and 32 because of the 3 color channels and 32 pixels. So for example you could use:**

        convo_1 = convolutional_layer(x,shape=[4,4,3,32])

In [24]:
convo_1 = convolutional_layer(x,shape=[4,4,3,32])
convo_1_pooling = max_pool_2by2(convo_1)

** Create the next convolutional and pooling layers.  The last two dimensions of the convo_2 layer should be 32,64 **

In [25]:
convo_2 = convolutional_layer(convo_1_pooling,shape=[4,4,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

** Now create a flattened layer by reshaping the pooling layer into [-1,8 \* 8 \* 64] or [-1,4096] **

In [26]:
8*8*64

4096

In [27]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1,8*8*64])

** Create a new full layer using the normal_full_layer function and passing in your flattend convolutional 2 layer with size=1024. (You could also choose to reduce this to something like 512)**

In [28]:
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

** Now create the dropout layer with tf.nn.dropout, remember to pass in your hold_prob placeholder. **

In [29]:
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

** Finally set the output to y_pred by passing in the dropout layer into the normal_full_layer function. The size should be 10 because of the 10 possible labels**

In [30]:
y_pred = normal_full_layer(full_one_dropout,10)

### Loss Function

** Create a cross_entropy loss function **

In [31]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

### Optimizer
** Create the optimizer using an Adam Optimizer. **

In [32]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

** Create a variable to intialize all the global tf variables. **

In [33]:
init = tf.global_variables_initializer()

## Graph Session

** Perform the training and test print outs in a Tf session and run your model! **

In [34]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(5000):
        batch = ch.next_batch(100)
        sess.run(train, feed_dict={x: batch[0], y_true: batch[1], hold_prob: 0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            print(sess.run(acc,feed_dict={x:ch.test_images,y_true:ch.test_labels,hold_prob:1.0}))
            print('\n')

Currently on step 0
Accuracy is:
0.0991


Currently on step 100
Accuracy is:
0.4162


Currently on step 200
Accuracy is:
0.4746


Currently on step 300
Accuracy is:
0.5017


Currently on step 400
Accuracy is:
0.5125


Currently on step 500
Accuracy is:
0.5276


Currently on step 600
Accuracy is:
0.5461


Currently on step 700
Accuracy is:
0.5688


Currently on step 800
Accuracy is:
0.5804


Currently on step 900
Accuracy is:
0.5662


Currently on step 1000
Accuracy is:
0.6098


Currently on step 1100
Accuracy is:
0.6118


Currently on step 1200
Accuracy is:
0.6083


Currently on step 1300
Accuracy is:
0.6193


Currently on step 1400
Accuracy is:
0.6234


Currently on step 1500
Accuracy is:
0.6414


Currently on step 1600
Accuracy is:
0.649


Currently on step 1700
Accuracy is:
0.6609


Currently on step 1800
Accuracy is:
0.6513


Currently on step 1900
Accuracy is:
0.6504


Currently on step 2000
Accuracy is:
0.6579


Currently on step 2100
Accuracy is:
0.6762


Currently on step 2200
